# The Array-Backed List

## Agenda

1. The List **Abstract Data Type** (ADT)
2. A List **Data Structure**
3. Our List API
4. Getting started: how to store our data?
5. Built-in `list` as array
6. The `ArrayList` data structure

## 1. The List **Abstract Data Type** (ADT)

An **abstract data type (ADT)** defines a *conceptual model* for how data may be stored and accessed.

A **list ADT** is a data container where:

- values are ordered in a *sequence*
- each value has at most one preceding and one succeeding value
- a given value may appear more than once in a list

Other common ADTs (some of which we'll explore later) include:

- Stacks
- Queues
- Priority Queues
- Maps
- Graphs

## 2. A List **Data Structure**

A **list data structure** is a *concrete implementation* of the list ADT in some programming language, which, in addition to adhering to the basic premises of the ADT, will also typically support operations that:

- access values in the list by their position (index)
- append and insert new values into the list
- remove values from the list

The implementation of any data structure will generally rely on simpler, constituent data types (e.g., "primitive" types offered by the language), the choice of which may affect the runtime complexities of said operations.

## 3. The List API

The operations we'll be building into our list data structures will be based on the [common](https://docs.python.org/3.6/library/stdtypes.html#common-sequence-operations) and [mutable](https://docs.python.org/3.6/library/stdtypes.html#mutable-sequence-types) sequence operations defined by the Python library.

In [ ]:
class List:        
    ### subscript-based access ###
    
    def __getitem__(self, idx):
        """Implements `x = self[idx]`"""
        pass

    def __setitem__(self, idx, value):
        """Implements `self[idx] = x`"""
        pass

    def __delitem__(self, idx):
        """Implements `del self[idx]`"""
        pass
    
    ### stringification ###
            
    def __repr__(self):
        """Supports inspection"""
        return '[]'
    
    def __str__(self):
        """Implements `str(self)`"""
        return '[]'

    ### single-element manipulation ###
    
    def append(self, value):
        pass
    
    def insert(self, idx, value):
        pass
    
    def pop(self, idx=-1):
        pass
    
    def remove(self, value):
        pass
    
    ### predicates (T/F queries) ###
    
    def __eq__(self, other):
        """Implements `self == other`"""
        return True

    def __contains__(self, value):
        """Implements `val in self`"""
        return True
    
    ### queries ###
    
    def __len__(self):
        """Implements `len(self)`"""
        return len(self.data)
    
    def min(self):
        pass
    
    def max(self):
        pass
    
    def index(self, value, i, j):
        pass
    
    def count(self, value):
        pass

    ### bulk operations ###

    def __add__(self, other):
        """Implements `self + other_array_list`"""
        return self
    
    def clear(self):
        pass
    
    def copy(self):
        pass

    def extend(self, other):
        pass

    ### iteration ###
    
    def __iter__(self):
        """Supports iteration (via `iter(self)`)"""
        pass

## 4. Getting started: how to store our data?

In [ ]:
class List:
    def append(self, value):
        pass
    
    def __getitem__(self, idx):
        """Implements `x = self[idx]`"""
        pass

    def __setitem__(self, idx, value):
        """Implements `self[idx] = x`"""
        pass
    
    def __repr__(self):
        """Supports inspection"""
        pass

## 5. Built-in `list` as array

To use the built-in list as though it were a primitive array, we will constrain ourselves to just the following APIs on a given list `lst`:

1. `lst[i]` for getting and setting values at an *existing, positive* index `i`
2. `len(lst)` to obtain the number of slots
3. `lst.append(None)` to grow the list by *one slot at a time*
4. `del lst[len(lst)-1]` to delete the last slot in a list

## 6. The `ArrayList` data structure

In [ ]:
class ArrayList:
    def __init__(self):
        self.data = []

    def append(self, value):
        pass

    def __getitem__(self, idx):
        """Implements `x = self[idx]`"""
        assert isinstance(idx, int)
        pass

    def __setitem__(self, idx, value):
        """Implements `self[idx] = x`"""
        assert isinstance(idx, int)
        pass

    def __delitem__(self, idx):
        """Implements `del self[idx]`"""
        assert isinstance(idx, int)
        pass
    
    def __len__(self):
        """Implements `len(self)`"""
        pass
    
    def __repr__(self):
        """Supports inspection"""
        pass